In [ ]:
#  !pip install -U "git+https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torch.utils.data as data
import random
import numpy as np
from torchsummary import summary

import copy
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
ROOT = '.data'
train_data = datasets.CIFAR10(root = ROOT, 
                              train = True, 
                              download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data


In [5]:
# Compute means and standard deviations along the R,G,B channel

means = train_data.data.mean(axis = (0,1,2)) / 255
stds = train_data.data.std(axis = (0,1,2)) / 255

In [6]:
from src.transforms import make_transforms

train_transforms, test_transforms = make_transforms(means, stds)

In [7]:
train_data = datasets.CIFAR10(ROOT, 
                              train = True, 
                              download = True, 
                              transform = train_transforms)

test_data = datasets.CIFAR10(ROOT, 
                             train = False, 
                             download = True, 
                             transform = test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [9]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [10]:
BATCH_SIZE = 256

train_iterator = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

valid_iterator = data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True)

test_iterator = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

**Define our Model

In [13]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

model_architecture = (
    (2, 64),
    (2, 128),
    (2, 196),
    (2, 196),
    (2, 128),
    (2, 128),
)

stem_config = StemConfig(num_channels=64, kernel_size=5, stride=1, padding=2)
model = ResNet(model_architecture, stem_config=stem_config, output_size=10)

Need to run a dummy set of data to initialize the lazy modules before we can use torchsummary

In [14]:
inputs = torch.empty((256, 3, 32, 32))
inputs.normal_()
model = model.to(device)
y = model(inputs)
y.size()

torch.Size([256, 10])

In [15]:
model.apply(initialize_parameters)

ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
  (body): Sequential(
    (block_2): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (out): ReLU(inplace=True)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1

In [16]:
print(f"num params: {sum([p.numel() for p in model.parameters() if p.requires_grad]):,}")

num params: 4,697,842


In [17]:
from src.engine import train_one_epoch, evaluate

best_loss = float('inf')
EPOCHS  = 10
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for epoch in range(1, EPOCHS+1):
    start = time.time()

    print(f"Epoch {epoch}")
    train_loss, train_acc = train_one_epoch(model, train_iterator, criterion, optimizer, device)
    train_mins, train_secs = epoch_time(start, time.time())

    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    start = time.time()
    val_loss, val_acc = evaluate(model, valid_iterator, criterion, device)
    val_mins, val_secs = epoch_time(start, time.time())

    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "resnet_alex.pt")

Epoch 1
	Train elapsed: 4:49, loss: 1.9210, acc: 34.81%
	Validation elapsed: 0:9, loss: 1.6242, acc: 40.96%
Epoch 2
	Train elapsed: 4:50, loss: 1.3680, acc: 50.13%
	Validation elapsed: 0:10, loss: 1.5030, acc: 47.65%
Epoch 3
	Train elapsed: 4:51, loss: 1.1502, acc: 58.71%
	Validation elapsed: 0:9, loss: 1.0909, acc: 60.90%
Epoch 4
	Train elapsed: 4:45, loss: 0.9724, acc: 65.20%
	Validation elapsed: 0:9, loss: 0.9474, acc: 66.04%
Epoch 5
	Train elapsed: 4:48, loss: 0.8493, acc: 69.66%
	Validation elapsed: 0:9, loss: 1.0575, acc: 65.24%
Epoch 6
	Train elapsed: 4:47, loss: 0.7440, acc: 73.48%
	Validation elapsed: 0:9, loss: 0.7455, acc: 73.35%
Epoch 7
	Train elapsed: 4:49, loss: 0.6671, acc: 76.35%
	Validation elapsed: 0:9, loss: 0.8062, acc: 73.61%
Epoch 8
	Train elapsed: 4:49, loss: 0.6080, acc: 78.50%
	Validation elapsed: 0:9, loss: 0.7162, acc: 75.68%
Epoch 9
	Train elapsed: 4:52, loss: 0.5570, acc: 80.34%
	Validation elapsed: 0:10, loss: 0.6312, acc: 78.20%
Epoch 10
	Train elapsed: 4

## Evaluate the Model  

In [20]:
model.load_state_dict(torch.load("resnet_alex.pt"))
test_loss, test_acc = evaluate(model.to(device), test_iterator, criterion, device)
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

Test Loss: 0.6363
Test Accuracy: 78.36%
